<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto Canada</font></h1>


Download all the dependencies that will be used.


In [164]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

#!conda install -c conda-forge geopy --yes 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

<a id='item1'></a>


# <span style='color:red'> PART I </span> 

## 1. Scrape the web for the data (on wiki) and Explore the Dataset


### In this section we will first scrape the Wikipedia page to get the Toronto data using the BeautifulSoup api.
The link of the wiki is: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [204]:
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
html_data=(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')).text
soup = BeautifulSoup(html_data, 'html.parser')

### From the soup , we'll get the table  which contains the postalcode, borough and the neighborhood
Then extract the values and store them in a dataframe df.

In [205]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    
    #Ignore empty values
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
      
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [215]:
#saving data in a csv file
#df.to_csv('toronto_data.csv')

#shape
print('\nDataset dimension',df.shape)

df.head(12)


Dataset dimension (103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# <span style='color:red'> PART II </span> 

<span style='color:red '> Given that the Geocoder package is very unreliable, we'll use the csv file that has the geographical coordinates of each postal code</span>: http://cocl.us/Geospatial_data
### Hence the following lines are put in comment.

---!pip install geocoder
import geocoder
lat_lng_coords = None
#### Test - loop until one gets the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  lat_lng_coords = g.latlng
  
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude,longitude)

## Let's therefore load the geographical coordinates csv file

In [217]:
llg_data=pd.read_csv('Geospatial_Coordinates.csv')
#llg_data.head()

#### Adding the geographical cordinates into the borough dataframe (df)

In [218]:
print('\ndf initial shape :', df.shape)


# add empty columns : latitude and longitude
df["Latitude"] = np.nan
df["Longitude"] = np.nan

pst= df["PostalCode"]

#Completing the borough data (df) with the latititude 
#and longitude from the geographical dataset (llg_data)
row_num = int(pst.shape[0])

for i in range (row_num):
    for row in llg_data.itertuples(index=False):
        
        #For every postalcode in df, get its corresponding ll from llg_data
        if row[0] == pst[i]:
            df["Latitude"][i] =  row[1]
            df["Longitude"][i] = row[2]
            

print('df final shape:',df.shape)
df.head(12)


df initial shape : (103, 5)
df final shape: (103, 5)


<ipython-input-218-818829d8c1c8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Latitude"][i] =  row[1]
<ipython-input-218-818829d8c1c8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Longitude"][i] = row[2]


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# <span style='color:red'> PART III </span> 

#### Geopy library to get the latitude and longitude values of Toronto.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>to_explorer</em>, as shown below.


In [209]:
address = 'TORONTO, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are  43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [171]:
# create map of Toronto using latitude and longitude values
tor_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tor_map)  
    
tor_map

<img src ="map_toronto.png" width = 400>


### Let's simply segment and cluster only the neighborhoods in <em>North York.


In [210]:
NorthYork_data = df[df['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


#### The geographical coordinates of North York.


In [211]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


#### Let's visualize  North York neighborhoods.


In [174]:
# create map of Manhattan using latitude and longitude values
NorthYork_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#323459',
        fill_opacity=0.7,
        parse_html=False).add_to(NorthYork_map )  
    
NorthYork_map 

### Now, we will utilize the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [212]:
CLIENT_ID = 'dummmycvffff' 
CLIENT_SECRET = 'frgtyuiz234E' 
VERSION = '20180605' 
LIMIT = 100

#### Exploring the second neighborhood in the dataframe.


The neighborhood's name.


In [176]:
NorthYork_data.loc[1, 'Neighborhood']

'Victoria Village'

Get the neighborhood's latitude and longitude values.


In [177]:
neighborhood_lat = NorthYork_data.loc[1, 'Latitude'] 
neighborhood_long = NorthYork_data.loc[1, 'Longitude'] 

neighborhood_name = NorthYork_data.loc[1, 'Neighborhood'] 

print('\n Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_long))


 Latitude and longitude values of Victoria Village are 43.725882299999995, -79.31557159999998.


#### Now, let's get the top 50 venues that are in "Victoria Village" within a radius of 500 meters.


We create the GET request URL(variable **url**).


In [213]:
LIMIT = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_long, 
    radius, 
    LIMIT)

#url 

Send the GET request and examine the resutls


In [220]:
results = requests.get(url).json()
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4c633acb86b6be9a61268e34',
   'name': 'Victoria Village Arena',
   'location': {'lat': 43.72348055545508,
    'lng': -79.31563520925143,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.72348055545508,
      'lng': -79.31563520925143}],
    'distance': 267,
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada']},
   'categories': [{'id': '4bf58dd8d48988d185941735',
     'name': 'Hockey Arena',
     'pluralName': 'Hockey Arenas',
     'shortName': 'Hockey',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_hockey_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4c633acb86b6be9a61268e34-0'},
 {'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general'

All the information is in the _items_ key. 
The **get_category_type** function is to extract the category of the venue


In [180]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In the cell below, we clean the json and structure it into a _pandas_ dataframe.


In [181]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # JSON flatten(to render a table view)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-181-f6fd6c80b37f>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # JSON flatten(to render a table view)


,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Portugril,Portuguese Restaurant,43.725819,-79.312785
2,Tim Hortons,Coffee Shop,43.725517,-79.313103
3,Pizza Nova,Pizza Place,43.725824,-79.312860


***Number of venues returned by Foursquare***


In [182]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore Neighborhoods in North York


**The following cell repeats the same process (applied above for a neighborhood)  for all the neighborhoods**


In [183]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### running the above function on each neighborhood and create a new dataframe called _ny_venues_.


In [184]:
# type your answer here
ny_venues = getNearbyVenues(names=NorthYork_data['Neighborhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude'],
                                  radius=500)

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Glencairn
Don Mills South
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [185]:
print(ny_venues.shape)
ny_venues.head()

(229, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Let's check how many venues were returned for each neighborhood


In [186]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",26,26,26,26,26,26
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Don Mills North,4,4,4,4,4,4
Don Mills South,19,19,19,19,19,19
Downsview Central,2,2,2,2,2,2
Downsview East,3,3,3,3,3,3
Downsview Northwest,4,4,4,4,4,4
Downsview West,6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues


In [187]:
print('\nThere are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))


There are 94 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [188]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column (as last col)
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.tail(10)

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
219,Downsview Northwest,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
220,Downsview Northwest,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
221,Downsview Northwest,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
222,York Mills West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
223,York Mills West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
224,York Mills West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
225,Willowdale West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
226,Willowdale West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
227,Willowdale West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
228,Willowdale West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


The new dataframe size.


In [189]:
ny_onehot.shape

(229, 95)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [190]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.076923,0.000000,0.0,0.000000,0.000000,0.000000,0.038462,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.038462,0.00,0.000000,0.076923,0.000000,0.000000,0.038462,0.00,0.000000,0.038462,0.00,0.000000,0.038462,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,0.038462,0.000000,0.038462,0.038462,0.0,0.000000,0.000000,0.000000,0.00,0.038462,0.00,0.000000,0.038462,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.038462,0.000000,0.038462,0.000000,0.038462,0.038462,0.038462,0.038462,0.000000,0.0,0.00,0.000000,0.000000,0.038462,0.00,0.038462,0.000000,0.038462,0.000000,0.038462,0.00,0.038462,0.000000,0.00,0.000000,0.00,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.250000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.038462,0.0,0.038462,0.000000,0.000000,0.00,0.000000,0.076923,0.038462,0.000000,0.000000,0.00,0.038462,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.038462,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.038462,0.038462,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.038462,0.000000,0.076923,0.000000,0.038462,0.038462,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.076923,0.000000,0.0,0.00,0.038462,0.000000,0.038462,0.00,0.076923,0.000000,0.000000,0.000000,0.000000,0.00,0.038462,0.038462,0.00,0.038462,0.00,0.000000,0.038462
3,Don Mills North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.250000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,

#### Let's confirm the new size


In [191]:
ny_grouped.shape

(23, 95)

#### Let's print each neighborhood along with the top 5 most common venues


In [192]:
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("|----"+hood+"----|")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

|----Bathurst Manor, Wilson Heights, Downsview North----|
           venue  freq
0           Bank  0.08
1    Coffee Shop  0.08
2  Deli / Bodega  0.04
3    Pizza Place  0.04
4      Pet Store  0.04


|----Bayview Village----|
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Café  0.25
3                 Bank  0.25
4            Juice Bar  0.00


|----Bedford Park, Lawrence Manor East----|
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2         Pizza Place  0.08
3      Sandwich Place  0.08
4       Women's Store  0.04


|----Don Mills North----|
                  venue  freq
0   Japanese Restaurant  0.25
1                   Gym  0.25
2  Caribbean Restaurant  0.25
3                  Café  0.25
4             Juice Bar  0.00


|----Don Mills South----|
         venue  freq
0  Coffee Shop  0.11
1   Restaurant  0.11
2          Gym  0.11
3  Art Gallery  0.05
4    Bike Shop  0.05


|----Downsview Central--

#### Let's put that into a _pandas_ dataframe


A function to sort the venues in descending order.

In [193]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

The new dataframe and display the top 10 venues for each neighborhood.


In [194]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Gas Station,Chinese Restaurant,Pet Store,Park,Bridal Shop,Mobile Phone Shop,Intersection,Ice Cream Shop
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Pizza Place,Women's Store,Pub,Cupcake Shop,Greek Restaurant,Grocery Store,Comfort Food Restaurant
3,Don Mills North,Caribbean Restaurant,Café,Gym,Japanese Restaurant,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
4,Don Mills South,Restaurant,Coffee Shop,Gym,Grocery Store,Italian Restaurant,Dim Sum Restaurant,Bike Shop,Beer Store,Clothing Store,Sandwich Place


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [195]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 4, 2, 3, 3, 3])

A new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [196]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = NorthYork_data

# merge ny_grouped with NorthYork_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Department Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Women's Store,Deli / Bodega,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Clothing Store,Accessories Store,Shoe Store,Vietnamese Restaurant,Gift Shop,Coffee Shop,Boutique,Furniture / Home Store,Bakery,Comfort Food Restaurant
3,M3B,North York,Don Mills North,43.745906,-79.352188,3.0,Caribbean Restaurant,Café,Gym,Japanese Restaurant,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
4,M6B,North York,Glencairn,43.709577,-79.445073,3.0,Pizza Place,Bakery,Japanese Restaurant,Italian Restaurant,Women's Store,Deli / Bodega,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping


**Finally, let's visualize the resulting clusters!**


In [197]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    try:
        clr_cluster=int(cluster)
    except:
        clr_cluster=0
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clr_cluster-1],
        fill=True,
        fill_color=rainbow[clr_cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [198]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0,ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Women's Store,Deli / Bodega,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
23,North York,0.0,Park,Pharmacy,Pizza Place,Coffee Shop,Women's Store,Cupcake Shop,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping


#### Cluster 2


In [199]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,1.0,Cafeteria,Women's Store,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop


#### Cluster 3


In [200]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Park,Food & Drink Shop,Department Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
11,North York,2.0,Park,Airport,Business Service,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


#### Cluster 4


In [201]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,3.0,Clothing Store,Accessories Store,Shoe Store,Vietnamese Restaurant,Gift Shop,Coffee Shop,Boutique,Furniture / Home Store,Bakery,Comfort Food Restaurant
3,North York,3.0,Caribbean Restaurant,Café,Gym,Japanese Restaurant,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
4,North York,3.0,Pizza Place,Bakery,Japanese Restaurant,Italian Restaurant,Women's Store,Deli / Bodega,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
5,North York,3.0,Restaurant,Coffee Shop,Gym,Grocery Store,Italian Restaurant,Dim Sum Restaurant,Bike Shop,Beer Store,Clothing Store,Sandwich Place
6,North York,3.0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Pool,Women's Store,Cupcake Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant
7,North York,3.0,Bank,Coffee Shop,Gas Station,Chinese Restaurant,Pet Store,Park,Bridal Shop,Mobile Phone Shop,Intersection,Ice Cream Shop
8,North York,3.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Juice Bar,Bank,Restaurant,Mobile Phone Shop,Miscellaneous Shop,Burger Joint
9,North York,3.0,Furniture / Home Store,Caribbean Restaurant,Coffee Shop,Massage Studio,Bar,Miscellaneous Shop,Deli / Bodega,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
10,North York,3.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
13,North York,3.0,Grocery Store,Hotel,Park,Bank,Shopping Mall,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping


#### Cluster 5


In [202]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,4.0,Food Truck,Baseball Field,Women's Store,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
19,North York,4.0,Construction & Landscaping,Baseball Field,Women's Store,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop


### Thank you !

This notebook is part of a course on **Coursera** called _Applied Data Science Capstone_. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).
